In [13]:
from collections import defaultdict

class Table:
    def __init__(self,columns):
        self.columns=columns
        self.rows=[]
    
    def __repr__(self):
        return str(self.columns)+ "\n"+"\n".join(map(str,self.rows))
    
    def insert(self,row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict=dict(zip(self.columns,row_values))
        self.rows.append(row_dict)
        
    #UPDATE users
    #SET num_friends=3
    #WHERE user_id=1:
    def update(self,updates,predicate):
        for row in self.rows:
            if predicate(row):
                for column,new_value in updates.iteritems():
                    row[column]=new_value

    #DELETE FROM users:
    #DELETE FROM users WHERE user_id=1;
    def delete(self,predicate=lambda row:True):
        self.rows=[row for row in self.rows if not(predicate(row))]
    
    #SELECT * FROM users;
    #SELECT * FROM users LINMIT 2;
    #SELECT user_id FROM users;
    #SELECT user_id FROM users WHERE name='Dunn';
    def select(self,keep_columns=None,additional_columns=None):
        if keep_columns is None:
            keep_columns=self.columns
        if additional_columns is None:
            additional_columns={}
        
        result_table=Table(keep_columns+additional_columns.keys())
        
        for row in self.rows:
            new_row=[row[column] for column in keep_columns]
            for column_name,calculation in additional_columns.iteritems():
                new_row.append(calculation(row))
            result_table.insert(new_row)
        return result_table
    
    def where(self,predicate=lambda row:True):
        where_table=Table(self.columns)
        where_table.rows=filter(predicate,self.rows)
        return where_table
    
    def limit(self,num_rows):
        limit_table=Table(self.columns)
        limit_table.rows=self.rows[:num_rows]
        return limit_table
    
    #SELECT LENGTH(name) as name_length,
    # MIN(user_id) AS min_user_id,
    # COUNT(*) AS num_users
    #FROM users
    #GROUP BY LENGTH(name);
    
    #SELECT SUBSTR(name,1,1) AS first_letter,
    # AVG(num_friends) AS ave_num_friends
    #FROM users
    #GROUP BY SUBSTR(name,1,1)
    #HAVING AVG(num_friends)>1;
    def group_by(self,group_by_columns,aggregates,having=None):
        grouped_rows=defaultdict(list)
        
        for row in self.rows:
            key=tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)
        
        result_table=Table(group_by_columns+aggregates.keys())
        
        for key,rows in grouped_rows.iteritems():
            if having is None or having(rows):
                new_row=list(key)
                for aggregate_name,aggregate_fn in aggregates.iteritems():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)
        return result_table
    
    #SELECT * FROM users
    #ORDER BY name
    #LIMIT 2;
    def order_by(self,order):
        new_table=self.select()
        new_table.rows.sort(key=order)
        return new_table
    
    def join(self,other_table,left_join=False):
        join_on_columns=[c for c in self.columns
                        if c in other_table.columns]
        
        additional_columns=[c for c in other_table.columns
                           if c not in join_on_columns]
        
        join_table=Table(self.columns+additional_columns)
        
        for row in self.rows:
            def is_join(other_row):
                return all(other_row[c]==row[c] for c in join_on_columns)
            other_rows=other_table.where(is_join).rows
            
            for other_row in other_rows:
                join_table.insert([row[c] for c in self.columns]+
                                 [other_row[c] for c in additional_columns])
            if left_join and not other_rows:
                join_table.insert([row[c] for c in self.columns]+
                                 [None for c in additional_columns])
        return join_table
    



In [14]:


users=[[0,"Hero",0],
       [1,"Dunn",2],
       [2,"Sue",3],
       [3,"Chi",3]]


#CREATE TABLE users (
#    user_id INT NOT NULL,
#    name VARCHAR(200),
#    num_friends INT);

#INSERT INTO users (user_id,name,num_friends) VALUES (0,"Hero",0);


users=Table(["user_id","name","num_friends"])
users.insert([0,"Hero",0])
users.insert([1,"Dunn",2])
users.insert([2,"Sue",3])
users.insert([3,"Chi",3])
users.insert([4,"Thor",3])
users.insert([5,"Clive",2])
users.insert([6,"Hicks",3])
users.insert([7,"Devin",2])
users.insert([8,"Kate",2])
users.insert([9,"Klein",3])
users.insert([10,"Jen",1])

users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

In [15]:
                    
users.update({'num_friends':3},lambda row:row['user_id']==1)

users.delete(lambda row:row["user_id"]==1)
#users.delete() #刪除所有資料

#SELECT * FROM users;
users.select()
#SELECT * FROM users LIMIT 2;
users.limit(2)
#SELECT user_id FROM users;
users.select(keep_columns=["user_id"])

#SELECT user_id FROM users WHERE name='Dunn';
users.where(lambda row: row["name"]=="Dunn")\
.select(keep_columns=["user_id"])

#SELECT LENGTH(name) AS name_length FROM users;
def name_length(row): return len(row["name"])
users.select(keep_columns=[],additional_columns={"name_length":name_length})



['name_length']
{'name_length': 4}
{'name_length': 3}
{'name_length': 3}
{'name_length': 4}
{'name_length': 5}
{'name_length': 5}
{'name_length': 5}
{'name_length': 4}
{'name_length': 5}
{'name_length': 3}

In [16]:
def min_user_id(rows): return min(row["user_id"] for row in rows)
stats_by_length=users.select(additional_columns={"name_length":name_length}).\
    group_by(group_by_columns=["name_length"],aggregates={"min_user_id":min_user_id,"num_users":len})

In [17]:
#CREATE TABLE user_interests (
#   user_id,INT NOT NULL,
#   interest VARCHAR(100) NOT NULL);
user_interests=Table(["user_id","interest"])
user_interests.insert([0,"SQL"])
user_interests.insert([0,"NoSQL"])
user_interests.insert([2,"SQL"])
user_interests.insert([2,"NoSQL"])


#SELECT users.name
#FROM users
#JOIN user_interests
#ON users.user_id=user_interests.user_id
#WHERE user_interests.interest ='SQL'

sql_users=users\
.join(user_interests)\
.where(lambda row: row["interest"]=="SQL")\
.select(keep_columns=["name"])

def count_interests(rows):
    return len([row for row in rows if row["interest"] is not None])

user_interest_counts=users.join(user_interests,left_join=True)\
.group_by(group_by_columns=["user_id"],aggregates={"num_interests":count_interests})

print user_interest_counts

['user_id', 'num_interests']
{'user_id': 0, 'num_interests': 2}
{'user_id': 2, 'num_interests': 2}
{'user_id': 8, 'num_interests': 0}
{'user_id': 3, 'num_interests': 0}
{'user_id': 9, 'num_interests': 0}
{'user_id': 4, 'num_interests': 0}
{'user_id': 10, 'num_interests': 0}
{'user_id': 5, 'num_interests': 0}
{'user_id': 6, 'num_interests': 0}
{'user_id': 7, 'num_interests': 0}


In [19]:
#SELECT MIN(user_id) AS min_user_id FROM
#(SELECT user_id FROM user_interests WHERE interest='SQL') sqlinterests;

likes_sql_user_ids=user_interests.where(lambda row:row["interest"]=="SQL")\
.select(keep_columns=['user_id'])

likes_sql_user_ids.group_by(group_by_columns=[],aggregates={"min_user_id": min_user_id})



['min_user_id']
{'min_user_id': 0}